# Practical Worksheet

In this worksheet, we will be working with a small dataset of hyponym-hypernym pairs. Hyponymy is the `is-a` relation. So we will have pairs like `(cat, mammal)` meaning 'A cat is a mammal'. The hyponym is the more specific term (e.g., cat) and the hypernym is the more general term (e.g., mammal). In this notebook you will:

1. (3 pts) Use Logical Neural Networks with a very small hyponym dataset to infer a set of facts. You will discuss the kinds of facts that you can infer and the limitations of the model as it is implemented
2. (5 pts) Set up a Logic Tensor Network to learn word embeddings and predicates that can model a larger hyponymy dataset.
3. (5 pts) Evaluate the effect of different axioms in the LTN system.
4. (2 pts) Query your model.


## Part 0. Setup
Create an environment and install python 3.12, numpy, pandas, and scikit-learn.

Install LNNs using `pip install git+https://github.com/IBM/LNN`

Install LTNs using `pip install LTNtorch`

Import packages as below.

In [1]:
import pandas as pd
import numpy as np
import torch
!pip install ltntorch
import ltn


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: C:\Users\thijn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Part 1. Inferring facts using Logical Neural Networks

In this first part, we will manually specify a very small dictionary of hyponym facts. We have three hyponyms and three non-hyponyms. The hyponymy relation is transitive, meaning that if $x$ is a hyponym of $y$ and $y$ is a hyponym of $z$, then $x$ should be a hyponym of $z$.

You will:

a. (1.5 pt) Set up a LNN model with suitable variables, a transitivity axiom, and hyponymy data.

b. (0.5 pt) Run inference over the model.

c. (1 pt) Inspect the output of the model and discuss whether the output is as expected.

In [2]:
# We first set up a small dictionary of hyponyms
!pip install git+https://github.com/IBM/LNN
from lnn import Fact

hyp_dict = {('cat', 'mammal'):Fact.TRUE,
            ('dog', 'mammal'):Fact.TRUE,
            ('mammal', 'animal'):Fact.TRUE,
            ('cat', 'dog'):Fact.FALSE,
            ('animal', 'mammal'):Fact.FALSE,
            ('mammal', 'dog'):Fact.FALSE,}

  Running command git clone --filter=blob:none --quiet https://github.com/IBM/LNN 'C:\Users\thijn\AppData\Local\Temp\pip-req-build-iuapkhup'
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if

  Cloning https://github.com/IBM/LNN to c:\users\thijn\appdata\local\temp\pip-req-build-iuapkhup
  Resolved https://github.com/IBM/LNN to commit 18ea03a52a79e6bbe8dada76e1ad9b320cd894d4
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jupyter_lsp-2.3.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.17.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab_server-2.28.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached bleach-6.3.0-py3-none-any.whl.metadata (31 kB)
  Using cached defusedxml-0.7

### Part 1a) (1.5 pts) Setting up the model.
Set up a LNN model with suitable predicates and variables, a transitivity axiom, and hyponymy data.

In [3]:
# Initialize an empty model
from lnn import Model
model = Model()
from lnn import Propositions, And, Implies, Iff, Fact, Model, Or
A, B, C, D, E = Propositions("A", "B", "C", "D", "E")
IMPLIES=Implies(A, B)
AND=And(C, D)
IFF=Iff(AND, E)
SENTENCE =And(IMPLIES, IFF)



In [4]:
# Create a predicate of arity 2 called Hyps and three variables x, y, z
## YOUR CODE HERE ##
from lnn import Predicate, Variable
Hyps = Predicate('Hyps', arity=2)
x = Variable('x')
y = Variable('y')
z = Variable('z')

In [5]:
# Create a logical rule that encodes the fact that the hyponymy relation is transitive
## YOUR CODE HERE ##
transitivity_rule = Implies(And(Hyps(x, y), Hyps(y, z)), Hyps(x, z))

In [6]:
# Add the knowledge and the data (the hyponymy dict) to the model and print.
## YOUR CODE HERE ##
model.add_knowledge(transitivity_rule)
model.add_data({Hyps: {
    ('cat', 'mammal'): Fact.TRUE,
    ('dog', 'mammal'): Fact.TRUE,
    ('mammal', 'animal'): Fact.TRUE,
    ('cat', 'dog'): Fact.FALSE,
    ('animal', 'mammal'): Fact.FALSE,
    ('mammal', 'dog'): Fact.FALSE,
}})
print(model)


Model()


### Part 1b) (0.5 pts) Inferring facts
Run inference over the model and print the output.

In [7]:
# Part 1b (0.5 pts) Run inference over the model and print the output 
## YOUR CODE HERE ##
model.infer()
model.print()   



***************************************************************************
                                LNN Model

OPEN Implies: ((Hyps(0, 1) ∧ Hyps(1, 2)) → Hyps(0, 2)) 
('dog', 'mammal', 'dog')                                    TRUE (1.0, 1.0)
('cat', 'cat', 'mammal')                                    TRUE (1.0, 1.0)
('dog', 'cat', 'animal')                                 UNKNOWN (0.0, 1.0)
('dog', 'animal', 'cat')                                 UNKNOWN (0.0, 1.0)
('cat', 'dog', 'cat')                                       TRUE (1.0, 1.0)
('mammal', 'animal', 'animal')                              TRUE (1.0, 1.0)
('dog', 'animal', 'mammal')                                 TRUE (1.0, 1.0)
('cat', 'cat', 'animal')                                 UNKNOWN (0.0, 1.0)
('cat', 'animal', 'cat')                                 UNKNOWN (0.0, 1.0)
('cat', 'dog', 'mammal')                                    TRUE (1.0, 1.0)
('cat', 'dog', 'animal')                                    TRUE

### Part 1c) (1 pt) Inspecting the output.

You should see that there are various facts whose truth value is unknown. 


Q1: Why can we not infer the truth value of all facts with the given database and axioms?




Q2: Suggest a suitable axiom to add to this system that would help to infer more facts. You do not need to implement the axiom.





Q1: Sometimes we do not knwo anything about all facts involved such as (mammel, mammel) and (mammel, cat) then we can not know (mammel, cat) since we have none of the fax and these are never mentioned so we cannot learn them either


Q2: Implies(Hyps(x,y), not(Hyps(y,x))) if one is in one group sach cat in mammel then mammel not in cat. This should help create more false examples. 

## Part 2 (5 pts) Building Embeddings with Logic Tensor Networks.
In this part, we will build a Logic Tensor Network to learn embeddings for the hyponyms. You will:

a. (1 pt) Describe why learning embeddings for the hyponyms is a suitable approach.

b. (1 pt) Set up a predicate for the hyponymy relation.

c. (1 pt) Train a simple network on the hyponymy task.

d. (2 pts) Assess satisfaction on the test set  and negative sample set


### Importing the data

Below, we import the data into pandas dataframes. Take a look at the data to familiarise yourself with the format. In each .csv file we have a list of word pairs. 
- In train_hypernyms we have the set of hypernym pairs we will train on. 
- In test_hypernyms we have the set of pairs we will test on. 
- In non_hypernyms we have a set of word pairs that are not hypernym pairs.

In [8]:
import pandas as pd

train_df = pd.read_csv('../data/train_hypernyms.csv')
test_df = pd.read_csv('../data/test_hypernyms.csv')
neg_df = pd.read_csv('../data/non_hypernyms.csv')


train_pairs = train_df.values
test_pairs = test_df.values
neg_pairs = neg_df.values

print("Training pairs:")
print(train_pairs[:5])

print("Testing pairs:")
print(test_pairs[:5])

print("Negative pairs:")
print(neg_pairs[:5])


Training pairs:
[['supermarket' 'commercial building']
 ['hand tool' 'tool']
 ['peach' 'fruit']
 ['pike' 'fish']
 ['nail gun' 'power tool']]
Testing pairs:
[['workshop' 'building']
 ['train' 'vehicle']
 ['pine' 'physical object']
 ['snare drum' 'physical object']
 ['grape' 'physical object']]
Negative pairs:
[['jigsaw' 'nail gun']
 ['temple' 'synagogue']
 ['double bass' 'banjo']
 ['turkey' 'turkey']
 ['crocodile' 'snake']]


### Part 2a. (1 pt) Learning Embeddings

When we use a logic tensor network, we can choose to use data from outside sources or to train embeddings within the network. We will be training embeddings. Do you think this is a suitable approach for this dataset? Why or why not?

YOUR ANSWER HERE
This is a suitable aproach for the data set since we do have embeddings in the data so we should either train or get an universal embedder but those already contain relation information. Giving a lot of unwanted information to the model.

Below, we will set up the vocabulary and the initial random word embeddings to be trained.

In [9]:
# Build a set of vocab by taking the union of the hyponyms and hypernyms
vocab = set(train_df.hyper.unique()).union(train_df.hypo.unique())

# Set the dimension of the vocab to 10
vocab_dim = 10

# Build a dictionary of word embeddings initialised randomly and set to be trainable.
word_embeddings = {word: ltn.Constant(torch.rand((vocab_dim,)), trainable=True) \
                   for word in vocab}



### Part 2b. (1 pt) Defining a predicate.
Define a predicate as a feed-forward NN with ELU and sigmoid activation functions and one hidden layer of size 16

In [10]:
# Define a feed-forward NN  with ELU and sigmoid activation functions and one hidden layer of size 16.
class ModelHyp(torch.nn.Module):
    def __init__(self):
        ## YOUR CODE HERE ##    
        super(ModelHyp, self).__init__()
        self.elu = torch.nn.ELU()
        self.softmax = torch.nn.Softmax(dim=1)
        self.fc1 = torch.nn.Linear(2 * vocab_dim, 16)
        self.fc2 = torch.nn.Linear(16, 1)
        self.sigmoid = torch.nn.Sigmoid()



    def forward(self, *x):
        # Specify the forward pass with ELU on the hidden layers and sigmoid on the output
        x = list(x)
        x = torch.cat(x, dim=1)
        ## YOUR CODE HERE ##
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    
# Wrap the feed-forward NN to make it an LTN predicate called Hyp
Hyp = ltn.Predicate(ModelHyp())
x = ltn.Variable('x', torch.stack([word_embeddings[word].value for word in vocab]))
y = ltn.Variable('y', torch.stack([word_embeddings[word].value for word in vocab]))

# Define connectives, quantifiers, and SatAgg
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

### Part 2c. (1 pt) Training the network

We set up a simple network in which we view our knowledge base as consisting just of those pairs in the training set. So our knowledge base states that for each word pair in the training set, this is a hyponym pair. We want to maximise the satisfaction over this knowledge base. To do this, we write a suitable axiom to aggregate the satisfaction of the hyponymy predicate over these pairs, and train the parameters of the network.

In [11]:
# We have to optimize the parameters of the predicate and also of the embeddings
params = list(Hyp.parameters()) +[i.value for i in word_embeddings.values()]
optimizer = torch.optim.Adam(params, lr=0.001)

# Set up a training loop for 300 epochs
for epoch in range(300):    
    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
    sat_agg = SatAgg(
# Implement one axiom which aggregates the satisfaction across the (x, y) in train_pairs
        ## YOUR CODE HERE ##
        *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in train_pairs]
        ,
        

        # Our list of hyponym pairs is in train_pairs.
        # We want to maximise the satisfaction gained by inputting the embeddings of those words into
        # our hyponymy predicate
        
        

    )
    
    loss = 1. - sat_agg
    loss.backward()
    optimizer.step()

    # Print metrics every 20 epochs of training
    if epoch % 20 == 0:
        print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")

c:\Users\thijn\miniconda3\Lib\site-packages\ltn\core.py:363: SyntaxWarning: invalid escape sequence '\d'
  """


RuntimeError: Expected all tensors to be on the same device, but got mat1 is on cuda:0, different from other tensors on cpu (when checking argument in method wrapper_CUDA_addmm)

### Part 2d (2 pts) Assessing the satisfaction on the test set

Calculate the satisfaction over the test set using SatAgg. Do you think the model is generalising well? Now calculate the satisfaction over the negative samples dataset. Is this a suitable satisfaction level? Why or why not?

YOUR ANSWER HERE

The model performs well on the test set, having a satisfaction of 0.99 which means that it has an almost perfect generelisability to the test set. This is the same for the negative emmbeddings as those also have a high score of 0.99

In [12]:
test_vocab = set(test_df.hyper.unique()).union(test_df.hypo.unique())
test_x = ltn.Variable('x', torch.stack([word_embeddings[word].value for word in test_vocab]))
test_y = ltn.Variable('y', torch.stack([word_embeddings[word].value for word in test_vocab]))
neg_vocab = set(neg_df.hyper.unique()).union(neg_df.hypo.unique())
neg_x = ltn.Variable('x', torch.stack([word_embeddings[word].value for word in neg_vocab]))
neg_y = ltn.Variable('y', torch.stack([word_embeddings[word].value for word in neg_vocab]))

In [13]:
satisfaction_test = SatAgg(*[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in test_pairs],)
satisfaction_neg = SatAgg(*[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in neg_pairs],)
print(f"the satisfaction of the test dataset is: {satisfaction_test}")

print(f"the satisfaction of the negative dataset is: {satisfaction_neg}")

RuntimeError: Expected all tensors to be on the same device, but got mat1 is on cuda:0, different from other tensors on cpu (when checking argument in method wrapper_CUDA_addmm)

## Part 3. (5 pts) Evaluate the effect of different axioms in the LTN system

In this part you will:

a. (2 pts) Retrain the model and evaluate the performance with negation included

b. (2 pts) Retrain the model and evaluate performance with transitivity included

c. (1 pt) Discuss the effect of the different axioms introduced.

### Part 3a. (2pts)  Retraining the model with negation
Reinitialise the model and retrain, including information from the `neg_pairs` dataset.

In [ ]:
# Reinitialise the model
Hyp = ltn.Predicate(ModelHyp())

In [ ]:
# Set up the parameters and optimizer
## YOUR CODE HERE ##
params = list(Hyp.parameters()) +[i.value for i in word_embeddings.values()]
optimizer = torch.optim.Adam(params, lr=0.001)


# Set up a training loop for 300 epochs
    ## YOUR CODE HERE ##

for epoch in range(300):
    
    
    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
        ## YOUR CODE HERE ##
        sat_agg = SatAgg(
        # Implement one axiom which aggregates the satisfaction across the (x, y) in train_pairs
        ## YOUR CODE HERE ##
        *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in train_pairs],
        

        # Implement one axiom which aggregates the satisfaction across the (x, y) in neg_pairs
        # Note that this statement should involve a negation.
        ## YOUR CODE HERE ##
        *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in neg_pairs],)

    # Calculate the loss and propagate backwards
    ## YOUR CODE HERE ##
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()

    # Print metrics every 20 epochs of training
    ## YOUR CODE HERE ##
        if epoch % 20 == 0:
            print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")

 epoch 0 | loss 0.5194828510284424 | Train Sat 0.4805171489715576
 epoch 20 | loss 0.3263695240020752 | Train Sat 0.6736304759979248
 epoch 40 | loss 0.14462226629257202 | Train Sat 0.855377733707428
 epoch 60 | loss 0.04430818557739258 | Train Sat 0.9556918144226074
 epoch 80 | loss 0.009901165962219238 | Train Sat 0.9900988340377808
 epoch 100 | loss 0.001838088035583496 | Train Sat 0.9981619119644165
 epoch 120 | loss 0.0003591179847717285 | Train Sat 0.9996408820152283
 epoch 140 | loss 0.0001322031021118164 | Train Sat 0.9998677968978882
 epoch 160 | loss 0.00010347366333007812 | Train Sat 0.9998965263366699
 epoch 180 | loss 0.00010031461715698242 | Train Sat 0.999899685382843
 epoch 200 | loss 0.00010001659393310547 | Train Sat 0.9998999834060669
 epoch 220 | loss 0.00010001659393310547 | Train Sat 0.9998999834060669
 epoch 240 | loss 0.00010001659393310547 | Train Sat 0.9998999834060669
 epoch 260 | loss 0.00010001659393310547 | Train Sat 0.9998999834060669
 epoch 280 | loss 0.

In [ ]:
# Calculate the satisfaction across the test dataset and the negated dataset
satisfaction_test = SatAgg(*[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in test_pairs])
satisfaction_neg = SatAgg(*[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in neg_pairs])
print(f"the satisfaction of the test dataset is: {satisfaction_test.item()}")

print(f"the satisfaction of the negative dataset is: {satisfaction_neg.item()}")

the satisfaction of the test dataset is: 0.9998999834060669
the satisfaction of the negative dataset is: 0.9998999834060669


### Part 3b. (2 pts) Retraining the model with transitivity

As we discussed in Part 1, the hyponymy relation is transitive. This should be reflected in the axioms. Reinitialise the model and add an axiom expressing the rule:

$\forall x, y, z Hyp(x, y) \land Hyp(y, z) \implies Hyp(x, z)$

Retrain the model and evaluate on the test and negated datasets.

In [ ]:
# Reinitialise the model
## YOUR CODE HERE ##
Hyp = ltn.Predicate(ModelHyp())

In [ ]:
# Set up the parameters and optimizer
## YOUR CODE HERE ##
params = list(Hyp.parameters()) +[i.value for i in word_embeddings.values()]
optimizer = torch.optim.Adam(params, lr=0.001)


# Set up a training loop for 300 epochs
## YOUR CODE HERE ##
for epoch in range(300):    
    # Create variables x_, y_, and z_, grounded with values from the `word_embeddings` dictionary
    ## YOUR CODE HERE ##
    x_ = ltn.Variable('x_', torch.stack([word_embeddings[word].value for word in vocab]))
    y_ = ltn.Variable('y_', torch.stack([word_embeddings[word].value for word in vocab]))
    z_ = ltn.Variable('z_', torch.stack([word_embeddings[word].value for word in vocab]))

    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
    ## YOUR CODE HERE ##
    sat_agg = SatAgg(
        
        #Positive instances of hyponymy
        ## YOUR CODE HERE ##
        *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in train_pairs],

        #Negative instances of hyponymy
        ## YOUR CODE HERE ##
        *[Not(Hyp(word_embeddings[x], word_embeddings[y])) for x, y in neg_pairs],
        
        # Transitivity axiom
        ## YOUR CODE HERE ##
        Forall([x_, y_, z_],
                Implies(And(Hyp(x_, y_), Hyp(y_, z_)), Hyp(x_, z_))

    ))
    # Calculate the loss and propagate backwards
    ## YOUR CODE HERE ##
    loss = 1. - sat_agg
    loss.backward()
    optimizer.step()

    # Print metrics every 20 epochs of training
    ## YOUR CODE HERE ##
    if epoch % 20 == 0:
        print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")

 epoch 0 | loss 0.5601631999015808 | Train Sat 0.4398368000984192
 epoch 20 | loss 0.48356544971466064 | Train Sat 0.5164345502853394
 epoch 40 | loss 0.4884018898010254 | Train Sat 0.5115981101989746
 epoch 60 | loss 0.5044569969177246 | Train Sat 0.4955430030822754
 epoch 80 | loss 0.46948033571243286 | Train Sat 0.5305196642875671
 epoch 100 | loss 0.4353257417678833 | Train Sat 0.5646742582321167
 epoch 120 | loss 0.4845607876777649 | Train Sat 0.5154392123222351
 epoch 140 | loss 0.44327878952026367 | Train Sat 0.5567212104797363
 epoch 160 | loss 0.36909210681915283 | Train Sat 0.6309078931808472
 epoch 180 | loss 0.4273664355278015 | Train Sat 0.5726335644721985
 epoch 200 | loss 0.38229233026504517 | Train Sat 0.6177076697349548
 epoch 220 | loss 0.2256374955177307 | Train Sat 0.7743625044822693
 epoch 240 | loss 0.4212794899940491 | Train Sat 0.5787205100059509
 epoch 260 | loss 0.34680402278900146 | Train Sat 0.6531959772109985
 epoch 280 | loss 0.13517379760742188 | Train Sa

In [ ]:
# Calculate the satisfaction across the test dataset and the negated dataset
satisfaction_test = SatAgg(*[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in test_pairs])
satisfaction_neg = SatAgg(*[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in neg_pairs])
print(f"the satisfaction of the test dataset is: {satisfaction_test.item()}")

print(f"the satisfaction of the negative dataset is: {satisfaction_neg.item()}")

the satisfaction of the test dataset is: 0.9955769181251526
the satisfaction of the negative dataset is: 0.3272130489349365


### Part 3c. (1 pt)  Evaluating the model
How has the satisfaction changed across the test set and the set of negative examples as you include different axioms? Why has this happened? Write a couple of sentences with your conclusions about the datasets and the model you have built. 

YOUR ANSWER HERE
The satisfaction is great on the test set meaning the model does understand transitivity over new data, however the model struggles with the negative examples on the transitivity data set, this migth happen since we do not have a lot of pure non transitive examples. meaning that the model does not see wrong examples often enough

## Part 4 (2 pts) Querying the model

One of the strengths of Logic Tensor Networks is that you are able to query the models you have built. In this part you will:

a. (0.5 pts) Define a logical statement that you expect to hold in your model.

b. (1 pt) Query the model.

c. (0.5 pts) Discuss your result.

### Part 4a. (0.5 pts) Defining a query

Thinking about the properties of hyponymy, give a logical statement that you would expect to hold in your model. The statement can be quite simple.

YOUR ANSWER HERE

Since no word is a hyponym of themselfs we can say that Hyp(x,y) -> Not(Hyp(y,x))

### Part 4b. (1 pt) Querying the model

Write a function that returns the satisfaction level of your logical statement and determine the satisfaction level.

In [ ]:
# this function returns the satisfaction level of your logical formula
def phi():
    # Create variables p, q, and r and initialize with the values from 'word_embeddings'
    ## YOUR CODE HERE ##
    p = ltn.Variable('p', torch.stack([word_embeddings[word].value for word in vocab]))
    q = ltn.Variable('q', torch.stack([word_embeddings[word].value for word in vocab]))
    # Return the truth value of phi
    ## YOUR CODE HERE ##
    return Forall([p, q], 
                  Implies(Hyp(p, q), Not(Hyp(q, p))))
                  

In [ ]:
# Evaluate phi

## YOUR CODE HERE ##
Phi = phi()
print(f"The evaluation of phi is: {Phi.value}")

The evaluation of phi is: 0.4188346862792969


### Part 4c. (0.5 pts) Discuss the results

Was the satisfaction value what you expected to see? Why or why not?


YOUR ANSWER HERE
The satisfaction is way lower than expected, since we now that now formula is hyponym of themself, we know that the formula should hold. However in this model it seems to have a satisfaction o arounf 0.4 which suggest it does not hold. The error migth ensue because the model is still bad at handeling the negative examples 

## Wrap up

In this worksheet, we looked at the hyponymy relation that can hold between words.

1. We used Logical Neural Networks with a very small hyponym dataset to infer a set of facts, and discussed the kinds of facts that you can infer and the limitations of the model as it is implemented.
2. We set up a Logic Tensor Network to learn word embeddings and predicates that can model a larger hyponymy dataset.
3. We evaluated the effect of different axioms in the LTN system.
4. And finally, you queried your model with new logical statements.

For another 15 points, you can extend this worksheet in a number of different ways. 

### Possible extensions

1. Use a new dataset for the task of inferring relationships over data.
2. Use the same dataset with a different model that we have covered in class. You could potentially use Logical Neural Networks, although they are a little slow.
3. Extend the investigation already started in this notebook. How do you expect the hyponymy relation to behave? Can you improve performance on novel queries?
4. Extend this investigation by including semantic information into the word embeddings from external sources.
5. Other ideas? Feel free to discuss with me!



The previous part of the note book highligthed the issue that the LTN does not generelisize well to novel formula's. The next part of the notebook will extend to on this by looking if the LTN is better capable of generelising to a novel formula, if it is trained on all subparts of the formula. And to what extend training on those subparts impact the formula. To show this we will train the model to understand a version of the transitivity formula; (Hyp(x,y) & Hyp(y,z)) -> NOT(hyp(z,x))

This research is set up as follows, 4 models will be trained on different formulas, one on only the hyp(x,y) relation, one on hyp(x,y) and not(hyp(x,y)), one will also learn the transitivi (Hyp(x,y) & Hyp(y,z))-> Hyp(x,z) next to to the previuosly mentioned formulas. And the last model will just learn the (Hyp(x,y) & Hyp(y,z)) -> NOT(hyp(z,x)) relation to function as baseline. All models will be trained 5 times to account for the variance in training.  IF possible the model will be trained on 3 different amount of epochs (100, 300, 500) to account for an increase of training times with more formulas.

In [ ]:
# Set up the parameters and optimizer
## YOUR CODE HERE ##
params = list(Hyp.parameters()) +[i.value for i in word_embeddings.values()]
optimizer = torch.optim.Adam(params, lr=0.001)
hyp_only = []
Hyp_neg_only = []
hyp_neg_transitivity = []
baseline_psi = []

def psi():
    # Create variables p, q, and r and initialize with the values from 'word_embeddings'
    ## YOUR CODE HERE ##
    p = ltn.Variable('p', torch.stack([word_embeddings[word].value for word in vocab]))
    q = ltn.Variable('q', torch.stack([word_embeddings[word].value for word in vocab]))
    r = ltn.Variable('r', torch.stack([word_embeddings[word].value for word in vocab]))
    # Return the truth value of psi
    ## YOUR CODE HERE ##
    return Forall([p, q, r], 
                  Implies(And(Hyp(p, q), Hyp(q, r)), Not(Hyp(r, p))))
# Set up a training loop for 300 epochs
## YOUR CODE HERE ##
# reinitialise the model
Hyp = ltn.Predicate(ModelHyp())
# training with only the positive hyponymy instances
for i in range(5):
    for epoch in range(300):    
    # Create variables x_, y_, and z_, grounded with values from the `word_embeddings` dictionary
    ## YOUR CODE HERE ##

    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
    ## YOUR CODE HERE ##
        sat_agg = SatAgg(
            
            #Positive instances of hyponymy
            ## YOUR CODE HERE ##
            *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in train_pairs],

        )
        # Calculate the loss and propagate backwards
        ## YOUR CODE HERE ##
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()

        # Print metrics every 100 epochs of training
        ## YOUR CODE HERE ##
        if epoch % 100 == 0:
            print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")
    psi_value = psi()
    hyp_only.append(psi_value.item())

# reinitialise the model
Hyp = ltn.Predicate(ModelHyp())
# training with both positive and negative hyponymy instances
for i in range(5):
    for epoch in range(300):    
    # Create variables x_, y_, and z_, grounded with values from the `word_embeddings` dictionary
    ## YOUR CODE HERE ##

    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
    ## YOUR CODE HERE ##
        sat_agg = SatAgg(
            
            #Positive instances of hyponymy
            *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in train_pairs],
            #Negative instances of hyponymy
            *[Not(Hyp(word_embeddings[x], word_embeddings[y])) for x, y in neg_pairs],
        )
        # Calculate the loss and propagate backwards
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        # Print metrics every 100 epochs of training
        if epoch % 100 == 0:
            print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")
    psi_value = psi()
    Hyp_neg_only.append(psi_value.item())
# reinitialise the model
Hyp = ltn.Predicate(ModelHyp())
# training with both positive and negative hyponymy instances and transitivity axiom
for i in range(5):
    for epoch in range(300):    
    # Create variables x_, y_, and z_, grounded with values from the `word_embeddings` dictionary
    ## YOUR CODE HERE ##
        x_ = ltn.Variable('x_', torch.stack([word_embeddings[word].value for word in vocab]))
        y_ = ltn.Variable('y_', torch.stack([word_embeddings[word].value for word in vocab]))
        z_ = ltn.Variable('z_', torch.stack([word_embeddings[word].value for word in vocab]))
    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
    ## YOUR CODE HERE ##
        sat_agg = SatAgg(
            
            #Positive instances of hyponymy
            *[Hyp(word_embeddings[x], word_embeddings[y]) for x, y in train_pairs],
            #Negative instances of hyponymy
            *[Not(Hyp(word_embeddings[x], word_embeddings[y])) for x, y in neg_pairs],
            # Transitivity axiom
            Forall([x_, y_, z_],
                    Implies(And(Hyp(x_, y_), Hyp(y_, z_)), Hyp(x_, z_))
        )
        )
        # Calculate the loss and propagate backwards
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        # Print metrics every 100 epochs of training
        if epoch % 100 == 0:
            print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")
    psi_value = psi()
    hyp_neg_transitivity.append(psi_value.item())
# reinitialise the model
Hyp = ltn.Predicate(ModelHyp())
# training with baseline: psi formula only
for i in range(5):
    for epoch in range(300):    
    # Create variables x_, y_, and z_, grounded with values from the `word_embeddings` dictionary
    ## YOUR CODE HERE ##
        p = ltn.Variable('p', torch.stack([word_embeddings[word].value for word in vocab]))
        q = ltn.Variable('q', torch.stack([word_embeddings[word].value for word in vocab]))
        r = ltn.Variable('r', torch.stack([word_embeddings[word].value for word in vocab]))
    # Set up a variable sat_agg which is the result of aggregating the truth values of all the axioms
    ## YOUR CODE HERE ##
        sat_agg = SatAgg(
            Forall([p, q, r], 
                  Implies(And(Hyp(p, q), Hyp(q, r)), Not(Hyp(r, p))))
        )
        # Calculate the loss and propagate backwards
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        # Print metrics every 100 epochs of training
        if epoch % 100 == 0:
            print(f" epoch {epoch} | loss {loss} | Train Sat {sat_agg}")
    psi_value = psi()
    baseline_psi.append(psi_value.item())
# Print the results
print("Hyp only:", hyp_only)
print("Hyp and Neg only:", Hyp_neg_only)
print("Hyp, Neg and Transitivity:", hyp_neg_transitivity)
print("Baseline psi only:", baseline_psi)

# plot the results
import matplotlib.pyplot as plt
plt.plot(hyp_only, label='Hyp only')
plt.plot(Hyp_neg_only, label='Hyp and Neg only')
plt.plot(hyp_neg_transitivity, label
='Hyp, Neg and Transitivity')   
plt.plot(baseline_psi, label='Baseline psi only')
plt.xlabel('Iteration')
plt.ylabel('Psi value')
plt.legend()
plt.show()
